In [0]:
from pyspark.sql import SparkSession

In [0]:
spark = SparkSession.builder.appName('IMSQL').getOrCreate()

In [0]:
df = spark.read.csv('incident_event_log.csv',inferSchema=True,header=True)

In [0]:
from pyspark.sql.functions import datediff,date_format,to_date,to_timestamp

In [0]:
df=df.withColumn('resolved_ts',to_timestamp(df.resolved_at, 'dd/MM/yyyy HH:mm')).\
                withColumn('opened_ts',to_timestamp(df.opened_at, 'dd/MM/yyyy HH:mm')).\
                withColumn('sys_created_ts',to_timestamp(df.sys_created_at, 'dd/MM/yyyy HH:mm')).\
                withColumn('sys_updated_ts',to_timestamp(df.sys_updated_at, 'dd/MM/yyyy HH:mm')).\
                withColumn('closed_ts',to_timestamp(df.closed_at, 'dd/MM/yyyy HH:mm')).\
                withColumn('resolved',to_date(df.resolved_at, 'dd/MM/yyyy HH:mm')).\
                withColumn('opened',to_date(df.opened_at, 'dd/MM/yyyy HH:mm')).\
                withColumn('sys_created',to_date(df.sys_created_at, 'dd/MM/yyyy HH:mm')).\
                withColumn('sys_updated',to_date(df.sys_updated_at, 'dd/MM/yyyy HH:mm')).\
                withColumn('closed',to_date(df.closed_at, 'dd/MM/yyyy HH:mm')).\
                withColumn('duration',datediff(to_date(df.resolved_at, 'dd/MM/yyyy HH:mm'),to_date(df.opened_at, 'dd/MM/yyyy HH:mm')))

In [0]:
df_unique_incidents=df.filter("incident_state=='Closed'").sort("sys_mod_count",ascending=False).dropDuplicates(["number"])

In [0]:
df_unique_incidents.createOrReplaceTempView("IM")

In [0]:
spark.sql("describe IM").show(50)

+--------------------+---------+-------+
 col_name|data_type|comment|
+--------------------+---------+-------+
 number| string| null|
 incident_state| string| null|
 active| boolean| null|
 reassignment_count| int| null|
 reopen_count| int| null|
 sys_mod_count| int| null|
 made_sla| boolean| null|
 caller_id| string| null|
 opened_by| string| null|
 opened_at| string| null|
 sys_created_by| string| null|
 sys_created_at| string| null|
 sys_updated_by| string| null|
 sys_updated_at| string| null|
 contact_type| string| null|
 location| string| null|
 category| string| null|
 subcategory| string| null|
 u_symptom| string| null|
 cmdb_ci| string| null|
 impact| string| null|
 urgency| string| null|
 priority| string| null|
 assignment_group| string| null|
 assigned_to| string| null|
 knowledge| boolean| null|
u_priority_confir...| boolean| null|
 notify| string| null|
 problem_id| string| null|
 rfc| string| null|
 vendor| string| null|
 caused_by| string| null|
 closed_code| string| null|
 resolved_by| string| null|
 resolved_at| string| null|
 closed_at| string| null|
 resolved_ts|timestamp| null|
 opened_ts|timestamp| null|
 sys_created_ts|timestamp| null|
 sys_updated_ts|timestamp| null|
 closed_ts|timestamp| null|
 resolved| date| null|
 opened| date| null|
 sys_created| date| null|
 sys_updated| date| null|
 closed| date| null|
 duration| int| null|
+--------------------+---------+-------+

#### 1. Top 5 people with most resolved incidents

In [0]:
spark.sql(" select resolved_by,count(number) as Incidents_Resolved from IM \
            group by resolved_by order by Incidents_Resolved desc limit 5").show()

+---------------+------------------+
 resolved_by|Incidents_Resolved|
+---------------+------------------+
 Resolved by 11| 3071|
 Resolved by 15| 2415|
Resolved by 103| 689|
Resolved by 177| 686|
 Resolved by 32| 597|
+---------------+------------------+

#### 2. Based on least average duration, find the top 5 people with maxmium number of incidents resolved

In [0]:
spark.sql(" select resolved_by,count(number) as Incidents_Resolved,mean(duration) as Average_Duration from IM \
            group by resolved_by order by Average_Duration asc,Incidents_Resolved desc limit 5").show()

+---------------+------------------+----------------+
 resolved_by|Incidents_Resolved|Average_Duration|
+---------------+------------------+----------------+
 Resolved by 10| 4| 0.0|
 Resolved by 94| 4| 0.0|
 Resolved by 26| 2| 0.0|
Resolved by 145| 2| 0.0|
Resolved by 219| 1| 0.0|
+---------------+------------------+----------------+

#### 3. People with maximum number of high impact incidents resolved

In [0]:
spark.sql(" select resolved_by,count(number) as Incidents_Resolved from IM \
            where impact = '1 - High' group by resolved_by order by Incidents_Resolved desc limit 5").show()

+---------------+------------------+
 resolved_by|Incidents_Resolved|
+---------------+------------------+
 Resolved by 98| 20|
Resolved by 137| 17|
 Resolved by 11| 15|
Resolved by 165| 13|
Resolved by 111| 12|
+---------------+------------------+

#### 4a. In each impact levels, find the person with most number of incidents resolved

In [0]:
spark.sql(" select impact,resolved_by,Incidents_Resolved from (\
            select impact,resolved_by,count(number) as Incidents_Resolved,\
            row_number() over (partition by impact order by count(number) desc) as row_number from IM \
            group by impact,resolved_by order by impact asc, Incidents_Resolved desc) as rows \
            where row_number = 1").show()

+----------+--------------+------------------+
 impact| resolved_by|Incidents_Resolved|
+----------+--------------+------------------+
 1 - High|Resolved by 98| 20|
2 - Medium|Resolved by 11| 3045|
 3 - Low|Resolved by 66| 194|
+----------+--------------+------------------+

#### 4b. In each urgency levels, find the person with most number of incidents resolved

In [0]:
spark.sql(" select urgency,resolved_by,Incidents_Resolved from (\
            select urgency,resolved_by,count(number) as Incidents_Resolved,\
            row_number() over (partition by urgency order by count(number) desc) as row_number from IM \
            group by urgency,resolved_by order by urgency asc, Incidents_Resolved desc) as rows \
            where row_number = 1").show()

+----------+---------------+------------------+
 urgency| resolved_by|Incidents_Resolved|
+----------+---------------+------------------+
 1 - High|Resolved by 166| 38|
2 - Medium| Resolved by 11| 3047|
 3 - Low| Resolved by 66| 195|
+----------+---------------+------------------+

#### 4c. In each priority levels, find the person with most number of incidents resolved

In [0]:
spark.sql(" select priority,resolved_by,Incidents_Resolved from (\
            select priority,resolved_by,count(number) as Incidents_Resolved,\
            row_number() over (partition by priority order by count(number) desc) as row_number from IM \
            group by priority,resolved_by order by priority asc, Incidents_Resolved desc) as rows \
            where row_number = 1").show()

+------------+---------------+------------------+
 priority| resolved_by|Incidents_Resolved|
+------------+---------------+------------------+
1 - Critical| Resolved by 98| 16|
 2 - High|Resolved by 166| 40|
3 - Moderate| Resolved by 11| 3040|
 4 - Low| Resolved by 66| 195|
+------------+---------------+------------------+

#### 5. Find each contact type as a percentage of total incidents

In [0]:
spark.sql(" select contact_type,count(number) as Incidents_Reported,\
            cast(count(number)*100/sum(count(number)) over() as decimal(4,2)) as Percentage \
            from IM group by contact_type").show()

+--------------+------------------+----------+
 contact_type|Incidents_Reported|Percentage|
+--------------+------------------+----------+
 Phone| 24688| 99.08|
 Email| 59| 0.24|
 Self service| 158| 0.63|
 IVR| 9| 0.04|
Direct opening| 4| 0.02|
+--------------+------------------+----------+

#### 6. On each priority level, find the percentage of incidents which made SLA and which did not.

In [0]:
spark.sql(" select priority,case when made_sla = 'false' then 'NO' else 'YES' end as made_sla,\
            count(number) as No_of_Incidents,\
            cast(count(number)*100/sum(count(number)) over(partition by priority) as decimal(4,2)) as Percentage \
            from IM group by priority,made_sla order by priority asc, made_sla desc").show()

+------------+--------+---------------+----------+
 priority|made_sla|No_of_Incidents|Percentage|
+------------+--------+---------------+----------+
1 - Critical| YES| 5| 1.85|
1 - Critical| NO| 265| 98.15|
 2 - High| YES| 2| 0.49|
 2 - High| NO| 406| 99.51|
3 - Moderate| YES| 15145| 64.54|
3 - Moderate| NO| 8321| 35.46|
 4 - Low| YES| 651| 84.11|
 4 - Low| NO| 123| 15.89|
+------------+--------+---------------+----------+

#### 7. Top 5 location with the maximum number of incidents reported

In [0]:
spark.sql(" select location,count(number) as Incidents_Reported from IM \
            group by location order by Incidents_reported desc limit 5").show()

+------------+------------------+
 location|Incidents_Reported|
+------------+------------------+
Location 204| 5554|
Location 161| 4002|
Location 143| 3276|
Location 108| 2140|
 Location 93| 1934|
+------------+------------------+

#### 8. Which category of issues missed meeting the SLA the most?

In [0]:
spark.sql(" select category,count(number) as No_Of_Incidents_missing_SLA from IM \
            where made_sla=false group by category order by No_Of_Incidents_missing_SLA desc limit 5").show()

+-----------+---------------------------+
 category|No_Of_Incidents_missing_SLA|
+-----------+---------------------------+
Category 46| 1254|
Category 26| 1017|
Category 53| 1009|
Category 42| 689|
Category 23| 505|
+-----------+---------------------------+